# This is just a notebook to visualise 1kHz filtered raw data

In [1]:
cd "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/"

\\10.69.168.1\crnldata\waking\audrey_hay\L1imaging\AnalysedMarch2023


In [2]:
import quantities as pq
import numpy as np
import neo
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
import scipy
from scipy import interpolate
from scipy import fftpack
from scipy import signal

import mmap
import xarray as xr
import dask.array as da
%matplotlib widget


from ephyviewer import mkQApp, MainViewer, TraceViewer

In [3]:
folder_base = Path('Gaelle/Baseline_recording/BlueLinesOK/session5/OpenEphys') 

In [4]:
def other_mmap(filename, shape, dtype, chunks):
    with open(filename, "rb") as fh:
        # `mmap` duplicates the file descriptor
        # `0` means map the full file
        mm = mmap.mmap(fh.fileno(), 0, access=mmap.ACCESS_READ)

    # coerce to NumPy array of expected type and shape
    a = np.asarray(mm).view(dtype).reshape(shape)

    # Don't call `asarray`
    # name=False to avoid hashing
    a_d = da.from_array(a, chunks=chunks, asarray=False, name=False)
    return a_d

Load RawDataChannelExtractedDS.npy

In [5]:
filename = folder_base / f'RawDataChannelExtractedDS.npy'
All = np.load(filename, mmap_mode= 'r')
All.shape

(653497, 32)

Visualise All

In [7]:
app = mkQApp()


sample_rate = 1000.
t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer()
view1 = TraceViewer.from_numpy(All, sample_rate, t_start, 'Signals')
win.add_view(view1)

#Parameters can be set in script
#view1.params['scale_mode'] = 'same_for_all'
view1.params['display_labels'] = True
#And also parameters for each channel

view1.by_channel_params['ch0', 'gain'] = 0.00002
view1.by_channel_params['ch1', 'gain'] = 0.00002
view1.by_channel_params['ch2', 'gain'] = 0.00002
view1.by_channel_params['ch3', 'gain'] = 0.00002
view1.by_channel_params['ch4', 'gain'] = 0.00002
view1.by_channel_params['ch5', 'gain'] = 0.00002
view1.by_channel_params['ch6', 'gain'] = 0.00002
view1.by_channel_params['ch7', 'gain'] = 0.00002
view1.by_channel_params['ch8', 'gain'] = 0.00002
view1.by_channel_params['ch9', 'gain'] = 0.00002
view1.by_channel_params['ch10', 'gain'] = 0.00002
view1.by_channel_params['ch11', 'gain'] = 0.00002
view1.by_channel_params['ch12', 'gain'] = 0.00002
view1.by_channel_params['ch13', 'gain'] = 0.00002
view1.by_channel_params['ch14', 'gain'] = 0.00002
view1.by_channel_params['ch15', 'gain'] = 0.00002
view1.by_channel_params['ch15', 'gain'] = 0.00002
view1.by_channel_params['ch15', 'gain'] = 0.00002
view1.by_channel_params['ch16', 'gain'] = 0.00002
view1.by_channel_params['ch17', 'gain'] = 0.00002
view1.by_channel_params['ch18', 'gain'] = 0.00002
view1.by_channel_params['ch19', 'gain'] = 0.00002
view1.by_channel_params['ch20', 'gain'] = 0.00002
view1.by_channel_params['ch21', 'gain'] = 0.00002
view1.by_channel_params['ch22', 'gain'] = 0.00002
view1.by_channel_params['ch23', 'gain'] = 0.00002
view1.by_channel_params['ch24', 'gain'] = 0.00002
view1.by_channel_params['ch25', 'gain'] = 0.00002
view1.by_channel_params['ch26', 'gain'] = 0.0002
view1.by_channel_params['ch27', 'gain'] = 0.0002
view1.by_channel_params['ch28', 'gain'] = 0.0002
view1.by_channel_params['ch29', 'gain'] = 0.0002
view1.by_channel_params['ch30', 'gain'] = 0.00002
view1.by_channel_params['ch31', 'gain'] = 0.00002

view1.by_channel_params['ch7', 'offset'] = 1.6
view1.by_channel_params['ch0', 'offset'] = 1.5
view1.by_channel_params['ch1', 'offset'] = 1.4
view1.by_channel_params['ch2', 'offset'] = 1.3
view1.by_channel_params['ch3', 'offset'] = 1.2
view1.by_channel_params['ch4', 'offset'] = 1.1
view1.by_channel_params['ch5', 'offset'] = 1
view1.by_channel_params['ch6', 'offset'] = 0.9
view1.by_channel_params['ch8', 'offset'] = 0.8
view1.by_channel_params['ch9', 'offset'] = 0.7
view1.by_channel_params['ch10', 'offset'] = 0.6
view1.by_channel_params['ch11', 'offset'] = 0.5
view1.by_channel_params['ch12', 'offset'] = 0.4
view1.by_channel_params['ch13', 'offset'] = 0.3
view1.by_channel_params['ch14', 'offset'] = 0.2
view1.by_channel_params['ch15', 'offset'] = 0.1
view1.by_channel_params['ch16', 'offset'] = 0.
view1.by_channel_params['ch17', 'offset'] = -0.1
view1.by_channel_params['ch18', 'offset'] = -0.2
view1.by_channel_params['ch19', 'offset'] = -0.3
view1.by_channel_params['ch20', 'offset'] = -0.4
view1.by_channel_params['ch21', 'offset'] = -0.5
view1.by_channel_params['ch22', 'offset'] = -0.6
view1.by_channel_params['ch23', 'offset'] = -0.7
view1.by_channel_params['ch24', 'offset'] = -0.8
view1.by_channel_params['ch25', 'offset'] = -0.9
view1.by_channel_params['ch26', 'offset'] = -1.
view1.by_channel_params['ch27', 'offset'] = -1.1
view1.by_channel_params['ch28', 'offset'] = -1.2
view1.by_channel_params['ch29', 'offset'] = -1.3
view1.by_channel_params['ch30', 'offset'] = -1.4
view1.by_channel_params['ch31', 'offset'] = -1.5


#Run
win.show()
app.exec_()


0